In [12]:
import pandas as pd
import numpy as np
import nltk
import re
import string
import utilities as ut



In [16]:
# DEBEMOS RECIBIR EL ID DE USUARIO COMO PARAMETRO DE ENTRADA Y EL NUMERO DE COINCIDENCIAS BUSCADAS
ID_usuario = 422
Top = 10

In [17]:
database=pd.read_csv("database.csv")

In [18]:
# Ejemplo de recomendación de usuarios

def RecomendacionUsuario(Top, ID_usuario):
    ID_usuario = ID_usuario - 1
    df = database.copy()
    RecomendacionUsuario = ut.carga_datos(database).iloc[(-ut.carga_datos(database).iloc[:, ID_usuario]).argsort()[1:(Top+1)].values, ID_usuario]
    RecomendacionUsuario.index

    filtrado = pd.DataFrame()
    for id in RecomendacionUsuario.index:
        filtrado = filtrado.append(df[df["user_id"]== id])
    return filtrado



In [19]:
RecomendacionUsuario(Top, ID_usuario)

,user_id,user_name,user_surname,email,password_,year_birth,gender,mother_tongue,years_in,studies,working,support_type,expert,area,about_me,date,country,pic
947,948,Alan,Angelia,alan.angelia7@mail.com,LhdceDPLw,1985,Masculino,Español,Más de 2 años,FP (Formacion Profesional),No especificar,Orientacion laboral,No,Sierra del Rincon,"Musica, Juegos, Baile",22/11/2022,NaN,948.jpg
144,145,Lawrence,Robert,lawrence.robert8@yahoo.com,emeDVNyIt,1980,Femenino,Español,Más de 2 años,Grado universitario,No especificar,Orientacion sobre tramites,No,Sierra de la Cabrera,"Juegos, Musica, Cocina",17/01/2022,NaN,145.jpg
664,665,Pearl,Michael,pearl.michael9@gmail.com,gfLvXjKDd,1983,No especificar,Español,De 6 meses a 1 año,FP (Formacion Profesional),Si,Orientacion sobre temas legales,Si,Valle del Jarama,"Deportes, Juegos, Pintura",10/02/2021,NaN,665.jpg
551,552,Daron,Gary,daron.gary2@yahoo.com,PEtGWnAOX,1980,No especificar,Chino,De 1 año a 2 años,FP (Formacion Profesional),No,Orientacion laboral,No,Sierra de la Cabrera,"Viajar, Fotografia, Cocina",07/07/2022,NaN,552.jpg
510,511,Mary,Jane,mary.jane8@gmail.com,zTKNngqGC,1992,Femenino,Aleman,De 6 meses a 1 año,FP (Formacion Profesional),No especificar,Orientacion laboral,No,Sierra de la Cabrera,"Juegos, Baile, Cine",22/09/2021,NaN,511.jpg
846,847,Freeman,Daniel,freeman.daniel5@outlook.com,BDdWSIssE,1972,No especificar,Español,De 6 meses a 1 año,Master,No especificar,Emocional,Si,Sierra de la Cabrera,"Cocina, Juegos, Cine",13/04/2021,NaN,847.jpg
552,553,Glenn,Verna,glenn.verna7@yahoo.com,mFKvFKICT,1981,Masculino,"Arabe, Español",Más de 2 años,FP (Formacion Profesional),Si,Orientacion sobre temas legales,No,Valle Alto del Lozoya,"Baile, Cocina, Pintura",19/03/2022,NaN,553.jpg
167,168,Agnes,Thomas,agnes.thomas1@hotmail.com,duSZytSEd,1981,Masculino,Chino,Más de 2 años,FP (Formacion Profesional),No especificar,Emocional,Si,Valle Medio del Lozoya,"Cine, Juegos, Baile",13/03/2021,NaN,168.jpg
750,751,Carol,Susan,carol.susan1@hotmail.com,dGoNNSxee,1985,Masculino,"Ucraniano, Portugues",Menos de 6 meses,FP (Formacion Profesional),No especificar,Orientacion sobre temas legales,No,Sierra de la Cabrera,"Viajar, Lectura, Cocina",15/01/2022,NaN,751.jpg
103,104,Alice,Mildred,alice.mildred4@mail.kz,uOQUBjpgv,1985,Masculino,"Chino, Frances",De 6 meses a 1 año,FP (Formacion Profesional),No especificar,Emocional,Si,Sierra de la Cabrera,"Viajar, Pintura, Cocina",21/08/2021,NaN,104.jpg


In [20]:
def RecomendacionExperto(Top, ID_usuario):
    df = ut.automatizacion(ID_usuario, database).copy()
    
    RecomendacionUsuario = ut.carga_datos_expertos(df)[ID_usuario].sort_values(ascending=False)[1:Top]
    RecomendacionUsuario.index

    filtrado_expertos = pd.DataFrame()
    for id in RecomendacionUsuario.index:
        filtrado_expertos = filtrado_expertos.append(df[df["user_id"]== id])
    return filtrado_expertos

In [21]:
RecomendacionExperto(Top, ID_usuario)

AttributeError: module 'utilities' has no attribute 'automatizacion'